In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, GRU, Dropout, BatchNormalization, Conv1D, MaxPooling1D, Flatten, Input, Reshape, Conv2D, ReLU, MaxPool2D, Masking
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from tensorflow.keras.preprocessing.sequence import pad_sequences
import ast

# Load and preprocess the data
# Replace 'train.csv' with the actual path to your dataset
data = pd.read_csv('dataset/train.csv', header = None, converters = {
    3: ast.literal_eval,
    4: ast.literal_eval,
    5: ast.literal_eval
}, skiprows = 1)

df = pd.DataFrame()

df['acc_x'] = data[3]
df['acc_y'] = data[4]
df['acc_z'] = data[5]
df['gesture'] = data[2]

#remove invalid rows
df.drop(df.loc[df['acc_x']==0].index, inplace=True)
df.drop(df.loc[df['acc_y']==0].index, inplace=True)
df.drop(df.loc[df['acc_z']==0].index, inplace=True)

df = df.dropna()

# Convert the lists into arrays
acc_x = df['acc_x'].values
acc_y = df['acc_y'].values
acc_z = df['acc_z'].values

# Combine all axes into a sequence of shape (timesteps, features)
sequences = [np.array([x, y, z]).T for x, y, z in zip(acc_x, acc_y, acc_z)]

# Pad sequences to the length of the longest sequence
padded_sequences = pad_sequences(sequences, maxlen = 32, padding='post', dtype='float32')

# Encode labels
labels = df['gesture'].values
label_encoder = LabelEncoder()
encoded_labels = label_encoder.fit_transform(labels)
categorical_labels = to_categorical(encoded_labels)


2025-01-10 14:42:35.435442: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-01-10 14:42:35.457295: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-01-10 14:42:35.457328: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-01-10 14:42:35.457922: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-01-10 14:42:35.462165: I tensorflow/core/platform/cpu_feature_guar

In [2]:
#DATA AUGMENTATION preprocessing

#adding noise
def add_noise(data, noise_level=0.05):
    return (data + np.random.normal(0, noise_level, data.shape)).astype(np.float32)

# Original data: `x_train` (accelerometer sequences), `y_train` (labels)

padded_sequences = np.concatenate((padded_sequences, add_noise(padded_sequences)))
categorical_labels = np.concatenate((categorical_labels, categorical_labels))




def scale_data(data, scaling_factor=0.1):
    """
    Scale the data by a random factor.
    Args:
        data: Numpy array of shape (time_steps, 3).
        scaling_factor: Max scaling factor variation.
    Returns:
        Scaled data.
    """
    factor = 1 + np.random.uniform(-scaling_factor, scaling_factor)
    return (data * factor).astype(np.float32)

padded_sequences = np.concatenate((padded_sequences, scale_data(padded_sequences)))
categorical_labels = np.concatenate((categorical_labels, categorical_labels))





In [3]:
# Split the data
X_train, X_validation, y_train, y_validation = train_test_split(
    padded_sequences, categorical_labels, test_size=0.2, random_state=42
)

X_train, X_test, y_train, y_test = train_test_split(
    X_train, y_train, test_size=0.25, random_state=42)


In [4]:

# Neural network model
model = Sequential([
    Input(shape=(32, 3)),
    # Mask padding values
    #Masking(mask_value=0.0, input_shape=(None, 3)),
    Reshape((8,4,3)),
    Conv2D(filters=32, kernel_size = (3,3), padding = "same"),
    Dropout(0.3),
    BatchNormalization(),
    ReLU(),
    Conv2D(filters=16, kernel_size = (3,3), padding= "same"),
    Dropout(0.4),
    BatchNormalization(),
    ReLU(),
    MaxPool2D(pool_size = 2),
    
    
    Flatten(),
    Dense(16),
    Dropout(0.5),
    BatchNormalization(),
    ReLU(),

    Dense(len(label_encoder.classes_), activation = 'softmax'),
])

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
history = model.fit(
    X_train, y_train,
    validation_data=(X_validation, y_validation),
    epochs=50,
    batch_size=32,
    verbose=1
)

# Evaluate the model
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print(f"Test Loss: {test_loss}, Test Accuracy: {test_accuracy}")

# Save the model
model.save('cnn_gesture_classification_model.h5')

# Decode predicted labels for interpretability
predicted_classes = label_encoder.inverse_transform(np.argmax(model.predict(X_test), axis=1))


Epoch 1/50
214/214 [==============================] - 2s 4ms/step - loss: 2.7899 - accuracy: 0.1355 - val_loss: 2.5484 - val_accuracy: 0.3596
Epoch 2/50
214/214 [==============================] - 1s 3ms/step - loss: 2.3422 - accuracy: 0.2794 - val_loss: 2.1593 - val_accuracy: 0.6018
Epoch 3/50
214/214 [==============================] - 1s 3ms/step - loss: 2.0390 - accuracy: 0.3579 - val_loss: 1.7547 - val_accuracy: 0.7329
Epoch 4/50
214/214 [==============================] - 1s 3ms/step - loss: 1.8174 - accuracy: 0.4154 - val_loss: 1.3835 - val_accuracy: 0.7991
Epoch 5/50
214/214 [==============================] - 1s 3ms/step - loss: 1.6085 - accuracy: 0.4936 - val_loss: 1.0047 - val_accuracy: 0.8623
Epoch 6/50
214/214 [==============================] - 1s 3ms/step - loss: 1.4452 - accuracy: 0.5417 - val_loss: 0.7786 - val_accuracy: 0.8803
Epoch 7/50
214/214 [==============================] - 1s 3ms/step - loss: 1.3014 - accuracy: 0.5794 - val_loss: 0.6317 - val_accuracy: 0.8877
Epoch 

/home/amroset/anaconda3/envs/microcontrollers/lib/python3.10/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [8]:
#Inference
from tensorflow.keras.models import load_model

# Load the trained model
model = load_model('models/cnn_gesture_classification_model.h5')
model.summary()

print(padded_sequences)

# Load the test data
test_data = pd.read_csv('dataset/test.csv', header=None, converters={
    2: ast.literal_eval,
    3: ast.literal_eval,
    4: ast.literal_eval
}, skiprows=1)

# Preprocess the test data
df_test = pd.DataFrame()
df_test['acc_x'] = test_data[2]
df_test['acc_y'] = test_data[3]
df_test['acc_z'] = test_data[4]

# Ensure all data is consistent (dropping invalid or zero entries)
df_test.drop(df_test.loc[df_test['acc_x'] == 0].index, inplace=True)
df_test.drop(df_test.loc[df_test['acc_y'] == 0].index, inplace=True)
df_test.drop(df_test.loc[df_test['acc_z'] == 0].index, inplace=True)

df_test = df_test.dropna()

# Extract accelerometer data (acc_x, acc_y, acc_z) for the test set
acc_x = df_test['acc_x'].values
acc_y = df_test['acc_y'].values
acc_z = df_test['acc_z'].values

test_sequences = [np.array([x, y, z]).T for x, y, z in zip(acc_x, acc_y, acc_z)]

# Pad sequences to the length of the longest sequence
padded_test_sequences = pad_sequences(test_sequences, padding='post', dtype='float32')

# Make predictions
predictions = model.predict(padded_test_sequences)

# Decode predictions to gesture classes
predicted_classes = np.argmax(predictions, axis=1)

# If you used a LabelEncoder for training, decode the classes
# Replace 'label_encoder' with your encoder used during training
gesture_labels = label_encoder.inverse_transform(predicted_classes)

df_out = pd.DataFrame()
df_out['id'] = test_data[0]
df_out['gesture'] = gesture_labels


# Save the results
df_out.to_csv('cnn_test_predictions.csv', index=False)

print("Predictions saved to test_predictions.csv")

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 reshape (Reshape)           (None, 8, 4, 3)           0         
                                                                 
 conv2d (Conv2D)             (None, 8, 4, 32)          896       
                                                                 
 dropout (Dropout)           (None, 8, 4, 32)          0         
                                                                 
 batch_normalization (Batch  (None, 8, 4, 32)          128       
 Normalization)                                                  
                                                                 
 re_lu (ReLU)                (None, 8, 4, 32)          0         
                                                                 
 conv2d_1 (Conv2D)           (None, 8, 4, 16)          4624      
                                                        

In [9]:
def representative_data_gen():
  for input_value in tf.data.Dataset.from_tensor_slices(padded_sequences).batch(1).take(32):
    yield [input_value]

# Ensure input shape is fixed
model.build(input_shape=(None, 32, 3))  # Example: Fixed length 32 timesteps, 3 features

converter = tf.lite.TFLiteConverter.from_keras_model(model)
# Apply integer quantization
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.representative_dataset = representative_data_gen
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]  # Ensure compatibility
converter.inference_input_type = tf.uint8  # Optional: Set input type
converter.inference_output_type = tf.uint8  # Optional: Set output type

tflite_model = converter.convert()

# Save the model
with open('cnn_gesture_classification_model_integer.tflite', 'wb') as f:
    f.write(tflite_model)

print("Model with full integer quantization saved as gesture_classification_model_integer.tflite")

INFO:tensorflow:Assets written to: /tmp/tmprk3nz1yx/assets


INFO:tensorflow:Assets written to: /tmp/tmprk3nz1yx/assets


Model with full integer quantization saved as gesture_classification_model_integer.tflite


/home/amroset/anaconda3/envs/microcontrollers/lib/python3.10/site-packages/tensorflow/lite/python/convert.py:953: UserWarning: Statistics for quantized inputs were expected, but not specified; continuing anyway.
  warnings.warn(
2025-01-04 15:08:48.379928: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:378] Ignored output_format.
2025-01-04 15:08:48.379944: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:381] Ignored drop_control_dependency.
2025-01-04 15:08:48.380125: I tensorflow/cc/saved_model/reader.cc:83] Reading SavedModel from: /tmp/tmprk3nz1yx
2025-01-04 15:08:48.381758: I tensorflow/cc/saved_model/reader.cc:51] Reading meta graph with tags { serve }
2025-01-04 15:08:48.381787: I tensorflow/cc/saved_model/reader.cc:146] Reading SavedModel debug info (if present) from: /tmp/tmprk3nz1yx
2025-01-04 15:08:48.385305: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:388] MLIR V1 optimization pass is not enabled
2025-01-04 15:08:48.38

In [10]:
import os

# Convert the model to TFLite without quantization
converter = tf.lite.TFLiteConverter.from_keras_model(model)
fp_tflite_model = converter.convert()

# Save the model to disk
open("cnn_model_f32.tflite", "wb").write(fp_tflite_model)

# Show the model size for the non-quantized HDF5 model
fp_h5_in_kb = os.path.getsize('models/gesture_classification_model.h5') / 1024
print("HDF5 Model size without quantization: %d KB" % fp_h5_in_kb)

# Show the model size for the non-quantized TFLite model
fp_tflite_in_kb = os.path.getsize('models/cnn_model_f32.tflite') / 1024
print("TFLite Model size without quantization: %d KB" % fp_tflite_in_kb)

# Determine the reduction in model size
print("\nReduction in file size by a factor of %f" % (fp_h5_in_kb / fp_tflite_in_kb))

INFO:tensorflow:Assets written to: /tmp/tmp9r8qxzuc/assets


INFO:tensorflow:Assets written to: /tmp/tmp9r8qxzuc/assets


HDF5 Model size without quantization: 442 KB
TFLite Model size without quantization: 35 KB

Reduction in file size by a factor of 12.375984


2025-01-04 15:08:51.855770: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:378] Ignored output_format.
2025-01-04 15:08:51.855786: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:381] Ignored drop_control_dependency.
2025-01-04 15:08:51.855891: I tensorflow/cc/saved_model/reader.cc:83] Reading SavedModel from: /tmp/tmp9r8qxzuc
2025-01-04 15:08:51.857569: I tensorflow/cc/saved_model/reader.cc:51] Reading meta graph with tags { serve }
2025-01-04 15:08:51.857587: I tensorflow/cc/saved_model/reader.cc:146] Reading SavedModel debug info (if present) from: /tmp/tmp9r8qxzuc
2025-01-04 15:08:51.861636: I tensorflow/cc/saved_model/loader.cc:233] Restoring SavedModel bundle.
2025-01-04 15:08:51.904751: I tensorflow/cc/saved_model/loader.cc:217] Running initialization op on SavedModel bundle at path: /tmp/tmp9r8qxzuc
2025-01-04 15:08:51.916893: I tensorflow/cc/saved_model/loader.cc:316] SavedModel load for tags { serve }; Status: success: OK. Took 61001 m

In [11]:
interpreter = tf.lite.Interpreter(model_content=tflite_model)
interpreter.allocate_tensors()

input_type = interpreter.get_input_details()[0]['dtype']
print('input: ', input_type)
output_type = interpreter.get_output_details()[0]['dtype']
print('output: ', output_type)

# Show the model size for the 8-bit quantized TFLite model
tflite_quant_in_kb = os.path.getsize('models/cnn_gesture_classification_model_integer.tflite') / 1024
print("TFLite Model size with 8-bit quantization: %d KB" % tflite_quant_in_kb)


input:  <class 'numpy.uint8'>
output:  <class 'numpy.uint8'>
TFLite Model size with 8-bit quantization: 14 KB


INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


In [12]:
# Helper function to run inference on a TFLite model

test_sequence_indices = range(X_test.shape[0])

print(y_test)

def run_tflite_model(tflite_file, test_image_indices):
  global X_test

  # Initialize the interpreter
  interpreter = tf.lite.Interpreter(model_path=str(tflite_file))
  interpreter.allocate_tensors()

  input_details = interpreter.get_input_details()[0]
  output_details = interpreter.get_output_details()[0]

  predictions = []
  for i, test_sequence_index in enumerate(test_sequence_indices):
    test_sequence = X_test[test_sequence_index]
    test_label = y_test[test_sequence_index]

    if (test_sequence_index % 100 == 0):
      print("Evaluated on %d sequences." % test_sequence_index)

    # Check if the input type is quantized, then rescale input data to uint8
    if input_details['dtype'] == np.uint8:
      input_scale, input_zero_point = input_details["quantization"]
      test_sequence = test_sequence / input_scale + input_zero_point

    test_sequence = np.expand_dims(test_sequence, axis=0).astype(input_details["dtype"])
    interpreter.set_tensor(input_details["index"], test_sequence)
    interpreter.invoke()
    output = interpreter.get_tensor(output_details["index"])[0]

    predictions.append(np.argmax(output, axis=-1))

  return predictions


# Helper function to evaluate a TFLite model on all images
def evaluate_model(tflite_file, model_type):
  global X_test
  global y_test

  test_sequence_indices = range(X_train.shape[0])
  predictions = run_tflite_model(tflite_file, test_sequence_indices)
    
  labels = np.argmax(y_test, axis=1)

  accuracy = (np.sum(labels == predictions) * 100) / len(X_test)

  print('%s model accuracy is %.4f%% (Number of test samples=%d)' % (
      model_type, accuracy, len(X_test)))

    


[[0. 0. 1. ... 0. 0. 0.]
 [0. 0. 1. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [13]:
import pathlib
# this might take a few minutes (~ 1- 2 minutes)
# if it takes longer than that, I suggest to 
# restart the runtime and try again
# if the issue still persists, restart your computer
tflite_model_quant_int8_file = pathlib.Path('models/cnn_gesture_classification_model_integer.tflite')
tflite_model_quant_int8_model_type = "Full Post-Quantized INT8"

evaluate_model(tflite_model_quant_int8_file, tflite_model_quant_int8_model_type)

input_details = interpreter.get_input_details()[0]
input_scale, input_zero_point = input_details["quantization"]

print(input_scale)
print(input_zero_point)




Evaluated on 0 sequences.
Evaluated on 100 sequences.
Evaluated on 200 sequences.
Evaluated on 300 sequences.
Evaluated on 400 sequences.
Evaluated on 500 sequences.
Evaluated on 600 sequences.
Evaluated on 700 sequences.
Evaluated on 800 sequences.
Evaluated on 900 sequences.
Evaluated on 1000 sequences.
Evaluated on 1100 sequences.
Evaluated on 1200 sequences.
Evaluated on 1300 sequences.
Evaluated on 1400 sequences.
Evaluated on 1500 sequences.
Evaluated on 1600 sequences.
Evaluated on 1700 sequences.
Evaluated on 1800 sequences.
Evaluated on 1900 sequences.
Evaluated on 2000 sequences.
Evaluated on 2100 sequences.
Evaluated on 2200 sequences.
Full Post-Quantized INT8 model accuracy is 98.0702% (Number of test samples=2280)
0.026128482073545456
119


In [11]:
# we plot also the confusion matrix of the quantized model
tflite_model_quant_int8_pred = run_tflite_model(tflite_model_quant_int8_file, range(y_test.shape[0]))

# compute the accuracy of the quantized model
from sklearn.metrics import accuracy_score

labels = np.argmax(y_test, axis=1)

full_int8_accuracy = accuracy_score(labels, tflite_model_quant_int8_pred)
print("Full-precision model accuracy is %.4f%% (Number of test samples=%d)" % (test_accuracy * 100, len(y_test)))
print("Quantized model accuracy is %.4f%% (Number of test samples=%d)" % (full_int8_accuracy * 100, len(y_test)))

Evaluated on 0 sequences.
Evaluated on 100 sequences.
Evaluated on 200 sequences.
Evaluated on 300 sequences.
Evaluated on 400 sequences.
Evaluated on 500 sequences.
Evaluated on 600 sequences.
Evaluated on 700 sequences.
Evaluated on 800 sequences.
Evaluated on 900 sequences.
Evaluated on 1000 sequences.
Evaluated on 1100 sequences.
Evaluated on 1200 sequences.
Evaluated on 1300 sequences.
Evaluated on 1400 sequences.
Evaluated on 1500 sequences.
Evaluated on 1600 sequences.
Evaluated on 1700 sequences.
Evaluated on 1800 sequences.
Evaluated on 1900 sequences.
Evaluated on 2000 sequences.
Evaluated on 2100 sequences.
Evaluated on 2200 sequences.
Full-precision model accuracy is 98.9035% (Number of test samples=2280)
Quantized model accuracy is 96.1404% (Number of test samples=2280)


In [14]:
# Function: Convert some hex value into an array for C programming
def hex_to_c_array(hex_data, var_name):

    c_str = ''

    # Create header guard
    c_str += '#ifndef ' + var_name.upper() + '_H\n'
    c_str += '#define ' + var_name.upper() + '_H\n\n'

    # Add array length at top of file
    c_str += '\nstatic const unsigned int ' + var_name + '_len = ' + str(len(hex_data)) + ';\n'

    # Declare C variable
    c_str += 'static const unsigned char ' + var_name + '[] = {'
    hex_array = []
    for i, val in enumerate(hex_data) :

        # Construct string from hex
        hex_str = format(val, '#04x')

        # Add formatting so each line stays within 80 characters
        if (i + 1) < len(hex_data):
            hex_str += ','
        if (i + 1) % 12 == 0:
            hex_str += '\n '
        hex_array.append(hex_str)

    # Add closing brace
    c_str += '\n ' + format(' '.join(hex_array)) + '\n};\n\n'

    # Close out header guard
    c_str += '#endif //' + var_name.upper() + '_H'

    return c_str

In [15]:
c_model_name = 'q8_cnn'
# check if dir 'cfiles' exists, if not create it
if not os.path.exists('cfiles'):
    os.makedirs('cfiles')
# Write TFLite model to a C source (or header) file
with open('cfiles/' + c_model_name + '.h', 'w') as file:
    file.write(hex_to_c_array(tflite_model, c_model_name))

In [14]:
input_details = interpreter.get_input_details()[0]
input_scale, input_zero_point = input_details["quantization"]

# save the test data as numpy arrays
np.save('x_test_gestures.npy', (X_test / input_scale + input_zero_point).astype(np.uint8))
np.save('y_test_gestures.npy', (y_test.astype(np.uint8)))

# print the location of the files
print('Test image data location: ', os.path.abspath('x_test_gestures.npy'))
print('Test labels location: ', os.path.abspath('y_test_gestures.npy'))

Test image data location:  /home/amroset/Machine Learning on Microcontrollers/Project/x_test_gestures.npy
Test labels location:  /home/amroset/Machine Learning on Microcontrollers/Project/y_test_gestures.npy


In [35]:
# this might take a few minutes (~ 1- 2 minutes)
# if it takes longer than that, I suggest to 
# restart the runtime and try again
# if the issue still persists, restart your computer
tflite_model_quant_int8_qat_file = pathlib.Path('models/no_qkeras_gesture_qat_int8.tflite')
tflite_model_quant_int8_qat_type = "Full QAT INT8"

evaluate_model(tflite_model_quant_int8_qat_file, tflite_model_quant_int8_qat_type)
print("Full-precision model accuracy is %.4f%% (Number of test samples=%d)" % (test_accuracy * 100, len(X_train)))

Evaluated on 0 sequences.
Evaluated on 100 sequences.
Evaluated on 200 sequences.
Evaluated on 300 sequences.
Evaluated on 400 sequences.
Evaluated on 500 sequences.
Evaluated on 600 sequences.
Evaluated on 700 sequences.
Evaluated on 800 sequences.
Evaluated on 900 sequences.
Evaluated on 1000 sequences.
Evaluated on 1100 sequences.
Evaluated on 1200 sequences.
Evaluated on 1300 sequences.
Evaluated on 1400 sequences.
Evaluated on 1500 sequences.
Evaluated on 1600 sequences.
Evaluated on 1700 sequences.
Evaluated on 1800 sequences.
Evaluated on 1900 sequences.
Evaluated on 2000 sequences.
Evaluated on 2100 sequences.
Evaluated on 2200 sequences.
Full QAT INT8 model accuracy is 97.8070% (Number of test samples=2280)
Full-precision model accuracy is 96.8860% (Number of test samples=6840)


In [38]:
# evaluate the model on the test set
tflite_model_quant_int8_qat_file = "models/no_qkeras_gesture_qat_int8.tflite"
tflite_model_quant_int8_qat_model_type = "Quantized aware training model"
tflite_model_quant_int8_qat_pred = run_tflite_model(tflite_model_quant_int8_qat_file, range(X_train.shape[0]))


Evaluated on 0 sequences.
Evaluated on 100 sequences.
Evaluated on 200 sequences.
Evaluated on 300 sequences.
Evaluated on 400 sequences.
Evaluated on 500 sequences.
Evaluated on 600 sequences.
Evaluated on 700 sequences.
Evaluated on 800 sequences.
Evaluated on 900 sequences.
Evaluated on 1000 sequences.
Evaluated on 1100 sequences.
Evaluated on 1200 sequences.
Evaluated on 1300 sequences.
Evaluated on 1400 sequences.
Evaluated on 1500 sequences.
Evaluated on 1600 sequences.
Evaluated on 1700 sequences.
Evaluated on 1800 sequences.
Evaluated on 1900 sequences.
Evaluated on 2000 sequences.
Evaluated on 2100 sequences.
Evaluated on 2200 sequences.


In [37]:
# compute the accuracy of the model
full_qat_int8_accuracy = accuracy_score(labels, tflite_model_quant_int8_qat_pred)
print('Full QAT INT8 accuracy is %.4f%% (Number of test samples=%d)' % (full_qat_int8_accuracy * 100, len(y_test)))
print('Full-precision model accuracy is %.4f%% (Number of test samples=%d)' % (test_accuracy * 100, len(y_test)))

Full QAT INT8 accuracy is 97.8070% (Number of test samples=2280)
Full-precision model accuracy is 96.8860% (Number of test samples=2280)
